目标检测之Faster RCNN
===
Faster RCNN在Fast RCNN的基础上，将proposal的选取工作也放到了网络中，这样通过一个CNN网络，就可以同时生成Proposal(RPN层)，而且也可以同时进行分类和回归的训练
![image](Images/03/02/01_04_001.png)

# 1.总体架构
![image](Images/03/02/01_04_002.png)

# 2.核心网络架构
![image](Images/03/02/01_04_003.png)

# 3.整体流程
## 3.1.编程角度
- Dataset：数据，提供符合要求的数据格式（目前常用数据集是VOC和COCO）
- Extractor： 利用CNN提取图片特征features（原始论文用的是ZF和VGG16，后来人们又用ResNet101）
- RPN(Region Proposal Network): 负责提供候选区域rois（每张图给出大概2000个候选框）
- RoIHead： 负责对rois分类和微调。对RPN找出的rois，判断它是否包含目标，并修正框的位置和座标

## 3.2.步骤
- 提特征：图片（img）经过预训练的网络（Extractor），提取到了图片的特征（feature）
- Region Proposal：利用提取的特征（feature），经过RPN网络，找出一定数量的rois（region of interests）。
- 分类与回归：将rois和图像特征features，输入到RoIHead，对这些rois进行分类，判断都属于什么类别，同时对这些rois的位置进行微调。

# 4.详细实现

## 4.1.数据
Faster RCNN支持不同尺度的图片，对于每张图片，作如下处理
- 图片进行缩放，使得长边小于等于1000，短边小于等于600（至少有一个等于）。
- 对相应的bounding boxes 也也进行同等尺度的缩放。
- 图片位于0-255，BGR格式，并减去一个均值，使得图片像素的均值为0。

最终返回四个值
- images：3×H×W ，BGR三通道，宽W，高H
- bboxes：4×K , K个bounding boxes，每个bounding box的左上角和右下角的座标，形如（Y_min,X_min, Y_max,X_max）,第Y行，第X列。
- labels：K，对应K个bounding boxes的label（对于VOC取值范围为[0-19]）
- scale: 缩放的倍数, 原图$H' \times W'$被resize到了$H \times W$(scale=H/H') 

## 4.2.特征提取
使用了经过一些改进后的Vgg16，最后得到的特征图是$Channel \times \frac{H}{16} \times \frac{W}{16}$大小
## 4.3.RPN网络
Faster R-CNN最突出的贡献就在于提出了Region Proposal Network(RPN)代替了Selective Search，从而将候选区域提取的时间开销几乎降为0(2s -> 0.01s)

### 4.3.1.Anchor
Anchor是大小和尺寸固定的候选框。论文中用到的anchor有三种尺寸和三种比例，如下图所示，三种尺寸分别是小(蓝128)中(红256)大(绿512)，三个比例分别是1:1，1:2，2:1。3×3的组合总共有9种anchor
![image](Images/03/02/01_04_004.jpg)
于是在特征图上，每个像素点上都有9个anchors，对于一个$512 \times 62 \times 37$的feature map,那么就有$62 \times 37 \times 9$个anchor，约等于20000。

这种做法很像是暴力穷举，20000多个anchor，哪怕是蒙也能够把绝大多数的ground truth bounding boxes蒙中
### 4.3.2.RPN网络
![image](Images/03/02/01_04_005.jpg)
我们知道每张特征图上anchors的数量与特征图的大小有关，而Faster R-CNN支持不同尺度的图片，所以必然每张特征图的anchors数量不一样。

我们可以看到，RPN在特征提取网络输出的特征图的基础上，先增加了一个卷积，然后利用两个$1 \times 1$的卷积分叉，一部分进行二分类，一部分进行位置回归。

进行分类的卷积核通道数为9×2(9个anchor，每个anchor二分类，使用交叉熵损失)，进行回归的卷积核通道数为9×4(9个anchor，每个anchor有4个位置参数)。RPN是一个全卷积网络(fully convolutional network)，这样对输入图片的尺寸就没有要求了

接下来RPN需要从20000多个anchors中选择256个anchor进行分类和位置回归，过程如下:
1. 对于每一个ground truth bounding box(gt_bbox)，选择和它重叠度(IoU)最高的一个anchor作为正样本
2. 对于剩下的anchor，从中选择和任意一个gt_bbox重叠度超过0.7的anchor，作为正样本，正样本的数目不超过128个
3. 随机选择和gt_bbox重叠度小于0.3的anchor作为负样本。负样本和正样本的总数为256.所谓负样本，就是背景.

对于每个anchor, gt_label 要么为1（前景），要么为0（背景），而gt_loc则是由4个位置参数(tx,ty,tw,th)组成，这样比直接回归座标更好。

计算分类损失用的是交叉熵损失，而计算回归损失用的是Smooth_l1_loss. 在计算回归损失的时候，只计算正样本（前景）的损失，不计算负样本的位置损失。

https://github.com/SirLPS/roi_pooling
https://zhuanlan.zhihu.com/p/32404424